# Using EASI Sentinel-1 RTC Gamma0 data

This notebook will demonstrate how to load and use Sentinel-1 RTC Gamma0 data generated in EASI.

The processing uses SNAP-10 with a *graph processing tool (GPT)* xml receipe for RTC Gamma0 and its variants.

For most uses we recommend the smoothed 20 m product (`sentinel1_grd_gamma0_20m`).
We can process the 10 m products (`sentinel1_grd_gamma0_10m`, `sentinel1_grd_gamma0_10m_unsmooth`) on request. Please also ask if you wish to trial other combinations of the parameters.

## RTC Gamma0 product variants
| | sentinel1_grd_gamma0_20m | sentinel1_grd_gamma0_10m | sentinel1_grd_gamma0_10m_unsmooth |
|--|--|--|--|
| **DEM** | | | |
| copernicus_dem_30 | Y | Y | Y |
| Scene to DEM extent multiplier| 3.0 | 3.0 | 3.0 |
| **SNAP operator** | | | |
| Apply-Orbit-File | Y | Y | Y |
| ThermalNoiseRemoval | Y | Y | Y |
| Remove-GRD-Border-Noise | Y | Y | Y |
| Calibration | Y | Y | Y |
| SetNoDataValue | Y | Y | Y |
| Terrain-Flattening | Y | Y | Y |
| Speckle-Filter | Y | Y | N |
| Multilook | Y | Y | N |
| Terrain-Correction | Y | Y | Y |
| **Output** | | | |
| Projection | WGS84, epsg:4326 | WGS84, epsg:4326 | WGS84, epsg:4326 |
| Pixel resolution | 20 m | 10 m | 10 m |
| Pixel alignment</br>Area = top-left | Area | Area | Area |

## Units and conversions

- intensity = amplitude * amplitude
- amplitude = sqrt(intensity)
- dB = 10*log10(intensity)
- intensity = 10**(dB/10)

Two example functions for scalar values. Below in the notebook we define functions for xarray datasets/arrays.
```
import math
def intensity_to_db(x):
    return 10*math.log10(x)
def db_to_intensity(db):
    return math.pow(10, db/10.0)
```
Reference: https://forum.step.esa.int/t/what-stage-of-processing-requires-the-linear-to-from-db-command

## Set up

### Import required packages and functions

In [ ]:
# Basic plots
%matplotlib inline
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Common imports and settings
import os, sys
from pathlib import Path
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr

# Datacube
import datacube
from datacube.utils.rio import configure_s3_access
from datacube.utils import masking
from datacube.utils.cog import write_cog
# https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools
from dea_tools.plotting import display_map
from dea_tools.datahandling import mostcommon_crs

# EASI tools
import git
repo = git.Repo('.', search_parent_directories=True).working_tree_dir  # This gets the current repo directory. Alternatively replace with the easi-notebooks repo path in your home directory
if repo not in sys.path: sys.path.append(repo)
from easi_tools import EasiDefaults, xarray_object_size
from easi_tools.notebook_utils import mostcommon_crs, initialize_dask, localcluster_dashboard, heading

# Data tools
import hvplot.xarray
import cartopy.crs as ccrs
import numpy as np
from dask.diagnostics import ProgressBar
from scipy.ndimage import uniform_filter, variance
from skimage.filters import threshold_minimum

# Dask
from dask.distributed import Client
from dask_gateway import Gateway

### EASI environment

In [ ]:
easi = EasiDefaults('asia')

family = 'sentinel-1'
# product = this.product(family)
product = 'sentinel1_grd_gamma0_20m'
display(Markdown(f'Default {family} product for "{easi.name}": [{product}]({easi.explorer}/products/{product})'))

### Dask and ODC

In [ ]:
# Start dask cluster - this may take a few minutes
# cluster, client = initialize_dask(workers=2)
# display(client)
# dashboard_address = localcluster_dashboard(client=client, server=easi.hub)
# display(dashboard_address)

cluster, client = initialize_dask(use_gateway=True, workers=5)
display(client)

# ODC
dc = datacube.Datacube()
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

# List measurements
dc.list_measurements().loc[[product]]

## Choose an area of interest

In [ ]:
# Set your own latitude / longitude

# PNG
# latitude = (-4.26, -3.75)
# longitude = (144.03, 144.74)
# time = ('2020-01-01', '2020-05-31')

# Bangladesh
latitude = (21.5, 23.5)
longitude = (89, 90.5)
time = ('2024-05-01', '2024-06-10')

# Vietnam
# epsg:32648
# latitude = (9.1, 9.9)
# longitude = (105.6, 106.4)
# time = ('2024-01-01', '2024-09-10')

display_map(longitude, latitude)

## Load data

In [ ]:
data = dc.load(
    product = product, 
    latitude = latitude,
    longitude = longitude,
    time = time,
    dask_chunks = {'latitude':2048, 'longitude':2048},      # Dask chunk size
    group_by = 'solar_day',                    # Group by day method
)

display(xarray_object_size(data))
display(data)

## Prepare the data

In [ ]:
# intensity = amplitude * amplitude
# amplitude = sqrt(intensity)
# dB = 10*log10(intensity)
# intensity = 10**(dB/10)

import math
def intensity_to_db(x):
    return 10*math.log10(x)
def db_to_intensity(db):
    return math.pow(10, db/10.0)

# intensity_to_db(0.5) # = -3
# intensity_to_db(0.001) # = -30
# db_to_intensity(-30) # = 0.001
# db_to_intensity(-3) # = 0.5

db_to_intensity(-19) # = 0.0125

# https://docs.xarray.dev/en/stable/user-guide/dask.html#automatic-parallelization-with-apply-ufunc-and-map-blocks
# https://docs.xarray.dev/en/stable/generated/xarray.apply_ufunc.html#xarray.apply_ufunc
# https://docs.xarray.dev/en/stable/examples/apply_ufunc_vectorize_1d.html

# Apply numpy.log10 to the DataArray
# log10_data = xr.apply_ufunc(np.log10, data)

In [ ]:
# Optional filters

# Select time layers with at least 5% of valid pixels
selected = data.vv.count(dim=['latitude','longitude']).values / (data.sizes['latitude']*data.sizes['longitude']) >= 0.05

data = data.sel(time=selected).persist()

## Plot the data

In [ ]:
def make_image(ds, frame_height=300, **kwargs):
    defaults = dict(
        cmap="Greys_r",
        x = 'longitude', y = 'latitude',
        rasterize=True,
        geo=True,
        frame_height=frame_height,
    )
    defaults.update(**kwargs)
    return ds.hvplot.image(**defaults)

In [ ]:
# A single time layer for VV and VH, with linked axes

vvplot = make_image(data.vv.isel(time=1), clim=(0, 0.5), title=f'VV ({data.time.dt.strftime("%Y-%m-%d %H:%M:%S").values[0]})', clabel='Intensity')
vhplot = make_image(data.vh.isel(time=1), clim=(0, 0.1), title=f'VH ({data.time.dt.strftime("%Y-%m-%d %H:%M:%S").values[0]})', clabel='Intensity')
vvplot + vhplot

In [ ]:
# Make a dB plot

# vvplot = make_image(intensity_to_db(data.vv.isel(time=0)), clim=(-30, -3), title=f'VV ({data.time.dt.strftime("%Y-%m-%d %H:%M:%S").values[0]})', clabel='DB')
# vhplot = make_image(intensity_to_db(data.vh.isel(time=0)), clim=(-30, -1), title=f'VH ({data.time.dt.strftime("%Y-%m-%d %H:%M:%S").values[0]})', clabel='DB')
# vvplot + vhplot

In [ ]:
# Subplots for each time layer for VV, with linked axes

make_image(data.vv, clim=(0,0.5)).layout().cols(4)

## Make an RGB image

For an RGB visualization we use the ratio between VH and VV.

In [ ]:
# Add the vh/vv band
data['vh_vv'] = data.vh / data.vv

# Scale the measurements by their median so they have a similar range for visualization
# med = data / data.median(dim=['latitude','longitude'])
med = data

# Create an RGB array, and persist it on the dask cluster
rgb_ds = xr.concat([med.vv, med.vh, med.vh_vv], 'channel').rename('rgb').to_dataset().persist()

In [ ]:
# Plot the RGB
rgb_plot = rgb_ds.hvplot.rgb(
    bands='channel',
    groupby='time', rasterize=True,
    geo=True, # crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    title='RGB', frame_height=500,
)

rgb_plot  # + vv_plot + vh_plot

## Export to Geotiffs

Recall that to write a dask dataset to a file requires the dataset to be `.compute()`ed. This may result in a large memory increase on your JupyterLab node if the area of interest is large enough, which in turn may kill the kernel. If so then skip this step, choose a smaller area or find a different way to export data.

In [ ]:
# Make a directory to save outputs to
target = Path.home() / 'output'
if not target.exists(): target.mkdir()

def write_band(ds, varname):
    """Write the variable name of the xarray dataset to a Geotiff files for each time layer"""
    for i in range(len(ds.time)):
        date = ds[varname].isel(time=i).time.dt.strftime('%Y%m%d').data
        single = ds[varname].isel(time=i).compute()
        write_cog(geo_im=single, fname=f'{target}/example_sentinel-1_{varname}_{date}.tif', overwrite=True)
        
write_band(data, 'vv')
write_band(data, 'vh')
# write_band(rgb_da, 'rgb')